NOTE: Each time the Preprocessing.py is changed
the kernel must be re-started

load fixations file
import necessary packages

In [10]:
import pandas as pd
import datetime
# from datetime import datetime
import pytz # datetime and pytz is for converting the timestamp
from Preprocessing import Preprocessing

#file_path = 'data/fixations.csv'
#df = pd.read_csv(file_path, delimiter=";")
filename = 'fixations.csv'
delimiter = ','
df = Preprocessing.load_data_csv(filename, delimiter)

-- loading dataset ...
-- found file 
-- done


function:
convert the original time stamp to a readable format
@input:
timestamp: int
e.g. 1683879584205738074 
@output: string
e.g. 2023-05-12 10:19:44.205738074


In [11]:
def convert_time_stamp(timestamp:int) -> str:
    timestamp_seconds = timestamp // 10**9 # Convert nanoseconds to seconds 
    # timestamp_nanoseconds = timestamp % 10**9 # Extract remaining nanoseconds 

    # Convert to datetime object 
    dt = datetime.datetime.utcfromtimestamp(timestamp_seconds) # a UTC datetime object

    # Define the target time zone (UTC+2 for Europe)
    target_timezone = pytz.timezone('Europe/Amsterdam')

    # Localize the UTC datetime object to the target time zone
    localized_dt = pytz.utc.localize(dt).astimezone(target_timezone)

    # Normalize the datetime to handle DST adjustments
    normalized_dt = target_timezone.normalize(localized_dt)

    final_dt_string = normalized_dt.strftime("%Y-%m-%d %H:%M:%S") # + f".{timestamp_nanoseconds}"
    return final_dt_string

# auto = convert_time_stamp(df['start timestamp [ns]'][0]) # 1683879584205738074
# print(auto)


convert the timestamp
input: dataframe before conversion
output: dataframe after conversion

In [12]:
for index, row in df.iterrows():
    # update timestamp - start
    current_value = row['start timestamp [ns]'] # Access the value in the column for the current row, type: int    
    modified_value = convert_time_stamp(current_value) # Modify the value as needed
    df.at[index, 'start timestamp [ns]'] = modified_value  # Update the value in the column for the current row

    # update timestamp - end
    current_value = row['end timestamp [ns]']
    modified_value = convert_time_stamp(current_value)
    df.at[index, 'end timestamp [ns]'] = modified_value



view df after the timestamp conversion


In [13]:
df

,section id,recording id,fixation id,start timestamp [ns],end timestamp [ns],duration [ms],fixation x [px],fixation y [px],azimuth [deg],elevation [deg]
0,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,1,2023-05-12 10:19:44,2023-05-12 10:19:44,196,577.646,477.851,1.718600,3.229121
1,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,2,2023-05-12 10:19:44,2023-05-12 10:19:44,475,418.252,451.261,-10.260214,5.196947
2,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,3,2023-05-12 10:19:44,2023-05-12 10:19:45,232,583.694,536.609,2.170390,-1.179995
3,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,4,2023-05-12 10:19:45,2023-05-12 10:19:45,311,651.299,458.259,7.258155,4.688697
4,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,5,2023-05-12 10:19:45,2023-05-12 10:19:45,272,584.164,448.699,2.212494,5.416615
...,...,...,...,...,...,...,...,...,...,...
1902,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,1903,2023-05-12 10:39:19,2023-05-12 10:39:19,200,443.826,374.293,-8.418634,10.962983
1903,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,1904,2023-05-12 10:39:20,2023-05-12 10:39:20,208,411.314,381.008,-10.873152,10.435651
1904,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,1905,2023-05-12 10:39:20,2023-05-12 10:39:20,103,397.940,393.011,-11.864504,9.528996
1905,b84e95f2-6d8b-40e5-ab30-af27b867a839,4bbf62fc-71fc-407c-8dd6-d476f6542689,1906,2023-05-12 10:39:20,2023-05-12 10:39:20,528,456.456,364.362,-7.473265,11.714923


count the number of records between timestamp 1 and timestamp 2
e.g.
timestamp 1: 2023-05-12 10:19:44 (type: str)
timestamp 2: 2023-05-12 10:39:20 (type: str)
there are 3 records between these two timestamps (including timestamp 1 and 2)

In [13]:
def count_records_with_start_timestamp(time1:str, time2:str) -> int:
    index_a = df.loc[df['start timestamp [ns]'] == time1].index[0]
    index_b = df.loc[df['start timestamp [ns]'] == time2].index[0]
    return(index_b - index_a + 1)

In [18]:
timestamp1 = '2023-05-12 10:19:44'
timestamp2 = '2023-05-12 10:19:44'
dt1 = datetime.datetime.strptime(timestamp1, "%Y-%m-%d %H:%M:%S") # convert the strings to datetime objects
dt2 = datetime.datetime.strptime(timestamp2, "%Y-%m-%d %H:%M:%S")
print(dt1>= dt2)


True


save/store the df to a csv file

In [ ]:
Preprocessing.save_data_csv(df, 'fixations_readable_timestamp.csv')